### Metrics: 
1. Wave Attenuation Ratio (WAR)
    - For this, no averaging across rollouts is performed
    - A single standardized shock is applied `--stability` must be present.
2. Controller Acceleration Variation (CAV)
    - This is present in the `eval_metrics.py` code

### Notes:

- Desired velocity change for FS May need monitoring (set correctly for 220)
- PI can have failures, so have to turn ON render and manually check
- LACC needs a different shock time (for no shock change time at eval)
- Generated data will be saved in the folder `test_time_rollout`
- We apply shocks after the systems have been stabilized
- Default shock times are between 8000 and 11000 unless explicitly specified otherwise
- No need to do multiple rollouts for stability, it will be the same

### Questions:
- Are multiple rollours required for WAR? Can be used
- Manual inspection of the results is recommended

## Ring: Unstable tests
- i.e., `--shock` must be present

In [1]:
# set parameters here
NUM_ROLLOUTS = 10
LENGTH = 260
# WAR does not have a standard deviation higher that 1%, so 1 rollout is fine
NUM_ROLLOUTS_WAR = 1

### 1. IDM

In [ ]:
# CAV
!python classic.py --method idm --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render

In [ ]:
!python eval_metrics.py --method idm --save_plots

In [ ]:
# Stability shock is applied to IDM when the system is stable and SGT is not formed yet
# 150 is pretty early-on
!python classic.py --method idm --length $LENGTH --gen_emission --stability --shock_start_time 150 --shock_end_time 3710 --render

In [ ]:
# IDM is unstable (So there is no wave attenuation) - WAR does not apply
# But do we still need a value for reference (WAR = 0)
!python eval_plots.py --method idm --start_time 150 --end_time 3710

### 2. Single Vehicle Systems

__FS__

In [ ]:
# For CAV (first: get the data)
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR
!python classic.py --method fs --stability --length $LENGTH --gen_emission --render


In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__
- This requires verifying if the particular run was `SUCCESS` or `FAIL`
- i.e., the controller may fail to stabilize the system

In [ ]:
# For CAV (first: get the data)
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render

# Stable or not before shocks were applied
# 1. Stable 
# 2. Stable
# 3. Stable
# 4. Stable
# 5. Stable
# 6. Stable
# 7. Stable
# 8. Stable
# 9. Stable
# 10. 

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --gen_emission --render

In [ ]:
!python eval_plots.py --method pi 

__Wu et. al.__

In [ ]:
# For CAV (first get the data)
!python test_rllib.py ./Wu_et_al/Trained_policies/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50 \
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR
!python test_rllib.py ./Wu_et_al/Trained_policies/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS_WAR --render


In [ ]:
!python eval_plots.py --method wu 

__Ours (1x)__

In [ ]:
# For CAV (first get the data)
!python test_rllib.py ./Ours/Trained_policies/Single_agent/PPO_DensityAwareRLEnv-v0_2355c52c_2023-07-21_18-27-20qktx7e8o 290\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render

# The logs appear broken because extra runs were performed to setup recording and then abruptly interrupted

In [ ]:
!python eval_metrics.py --method ours --save_plots

In [ ]:

# Data for WAR
!python test_rllib.py ./Ours/Trained_policies/Single_agent/PPO_DensityAwareRLEnv-v0_2355c52c_2023-07-21_18-27-20qktx7e8o 290\
--method ours --shock --stability --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS_WAR --render

In [ ]:
!python eval_plots.py --method ours

### 3. Multiple Vehicle Systems

__BCM__

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS_WAR --render


In [ ]:
!python eval_plots.py --method bcm 

__LACC__

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts $NUM_ROLLOUTS_WAR --gen_emission --shock_start_time 11400 --shock_end_time 15000 --render


In [ ]:
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

__Ours (4x)__

In [ ]:
# For CAV (first get the data)
!python test_rllib.py ./Ours/Trained_policies/Multi_agent_4x/PPO_MultiAgentDensityAwareRLEnv-v1_7ae93312_2023-07-22_17-24-10ppil4or8 34\
--method ours4x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render


In [ ]:
!python eval_metrics.py --method ours4x --save_plots

__Ours (9x)__

### Test

In [ ]:
import json
config = {'batch_mode': 'truncate_episodes', 'callbacks': {'on_episode_end': None, 'on_episode_start': None, 'on_episode_step': None, 'on_postprocess_traj': None, 'on_sample_end': None, 'on_train_result': None}, 'clip_actions': True, 'clip_param': 0.3, 'clip_rewards': None, 'collect_metrics_timeout': 180, 'compress_observations': False, 'custom_resources_per_worker': {}, 'eager': False, 'eager_tracing': False, 'entropy_coeff': 0.01, 'entropy_coeff_schedule': [[0, 0.1], [500000, 0.02], [1000000, 0.01]], 'env': 'MultiAgentDensityAwareRLEnv-v1', 'env_config': {'flow_params': '{\n    "env": {\n        "additional_params": {\n            "max_accel": 1,\n            "max_decel": 1,\n            "ring_length": [\n                260,\n                260\n            ]\n        },\n        "clip_actions": false,\n        "evaluate": false,\n        "horizon": 4500,\n        "sims_per_step": 1,\n        "warmup_steps": 2500\n    },\n    "env_name": "flow.envs.multiagent.ring.density_aware_env.MultiAgentDensityAwareRLEnv",\n    "exp_tag": "density_aware_multiagent_ring",\n    "initial": {\n        "additional_params": {},\n        "bunching": 0,\n        "edges_distribution": "all",\n        "lanes_distribution": Infinity,\n        "min_gap": 0,\n        "perturbation": 0.0,\n        "shuffle": false,\n        "spacing": "uniform",\n        "x0": 0\n    },\n    "net": {\n        "additional_params": {\n            "lanes": 1,\n            "length": 260,\n            "resolution": 40,\n            "speed_limit": 30\n        },\n        "inflows": {\n            "_InFlows__flows": []\n        },\n        "osm_path": null,\n        "template": null\n    },\n    "network": "flow.networks.ring.RingNetwork",\n    "sim": {\n        "color_by_speed": false,\n        "emission_path": null,\n        "force_color_update": false,\n        "lateral_resolution": null,\n        "no_step_log": true,\n        "num_clients": 1,\n        "overtake_right": false,\n        "port": null,\n        "print_warnings": true,\n        "pxpm": 2,\n        "render": false,\n        "restart_instance": false,\n        "save_render": false,\n        "seed": null,\n        "show_radius": false,\n        "sight_radius": 25,\n        "sim_step": 0.1,\n        "teleport_time": -1,\n        "use_ballistic": false\n    },\n    "simulator": "traci",\n    "veh": [\n        {\n            "acceleration_controller": [\n                "RLController",\n                {}\n            ],\n            "car_following_params": {\n                "controller_params": {\n                    "accel": 2.6,\n                    "carFollowModel": "IDM",\n                    "decel": 4.5,\n                    "impatience": 0.5,\n                    "maxSpeed": 30,\n                    "minGap": 0.1,\n                    "sigma": 0.5,\n                    "speedDev": 0.1,\n                    "speedFactor": 1.0,\n                    "tau": 1.0\n                },\n                "speed_mode": 25\n            },\n            "initial_speed": 0,\n            "lane_change_controller": [\n                "SimLaneChangeController",\n                {}\n            ],\n            "lane_change_params": {\n                "controller_params": {\n                    "laneChangeModel": "LC2013",\n                    "lcCooperative": "1.0",\n                    "lcKeepRight": "1.0",\n                    "lcSpeedGain": "1.0",\n                    "lcStrategic": "1.0"\n                },\n                "lane_change_mode": 512\n            },\n            "num_vehicles": 1,\n            "routing_controller": [\n                "ContinuousRouter",\n                {}\n            ],\n            "veh_id": "rl_0"\n        },\n        {\n            "acceleration_controller": [\n                "RLController",\n                {}\n            ],\n            "car_following_params": {\n                "controller_params": {\n                    "accel": 2.6,\n                    "carFollowModel": "IDM",\n                    "decel": 4.5,\n                    "impatience": 0.5,\n                    "maxSpeed": 30,\n                    "minGap": 0.1,\n                    "sigma": 0.5,\n                    "speedDev": 0.1,\n                    "speedFactor": 1.0,\n                    "tau": 1.0\n                },\n                "speed_mode": 25\n            },\n            "initial_speed": 0,\n            "lane_change_controller": [\n                "SimLaneChangeController",\n                {}\n            ],\n            "lane_change_params": {\n                "controller_params": {\n                    "laneChangeModel": "LC2013",\n                    "lcCooperative": "1.0",\n                    "lcKeepRight": "1.0",\n                    "lcSpeedGain": "1.0",\n                    "lcStrategic": "1.0"\n                },\n                "lane_change_mode": 512\n            },\n            "num_vehicles": 1,\n            "routing_controller": [\n                "ContinuousRouter",\n                {}\n            ],\n            "veh_id": "rl_1"\n        },\n        {\n            "acceleration_controller": [\n                "RLController",\n                {}\n            ],\n            "car_following_params": {\n                "controller_params": {\n                    "accel": 2.6,\n                    "carFollowModel": "IDM",\n                    "decel": 4.5,\n                    "impatience": 0.5,\n                    "maxSpeed": 30,\n                    "minGap": 0.1,\n                    "sigma": 0.5,\n                    "speedDev": 0.1,\n                    "speedFactor": 1.0,\n                    "tau": 1.0\n                },\n                "speed_mode": 25\n            },\n            "initial_speed": 0,\n            "lane_change_controller": [\n                "SimLaneChangeController",\n                {}\n            ],\n            "lane_change_params": {\n                "controller_params": {\n                    "laneChangeModel": "LC2013",\n                    "lcCooperative": "1.0",\n                    "lcKeepRight": "1.0",\n                    "lcSpeedGain": "1.0",\n                    "lcStrategic": "1.0"\n                },\n                "lane_change_mode": 512\n            },\n            "num_vehicles": 1,\n            "routing_controller": [\n                "ContinuousRouter",\n                {}\n            ],\n            "veh_id": "rl_2"\n        },\n        {\n            "acceleration_controller": [\n                "RLController",\n                {}\n            ],\n            "car_following_params": {\n                "controller_params": {\n                    "accel": 2.6,\n                    "carFollowModel": "IDM",\n                    "decel": 4.5,\n                    "impatience": 0.5,\n                    "maxSpeed": 30,\n                    "minGap": 0.1,\n                    "sigma": 0.5,\n                    "speedDev": 0.1,\n                    "speedFactor": 1.0,\n                    "tau": 1.0\n                },\n                "speed_mode": 25\n            },\n            "initial_speed": 0,\n            "lane_change_controller": [\n                "SimLaneChangeController",\n                {}\n            ],\n            "lane_change_params": {\n                "controller_params": {\n                    "laneChangeModel": "LC2013",\n                    "lcCooperative": "1.0",\n                    "lcKeepRight": "1.0",\n                    "lcSpeedGain": "1.0",\n                    "lcStrategic": "1.0"\n                },\n                "lane_change_mode": 512\n            },\n            "num_vehicles": 1,\n            "routing_controller": [\n                "ContinuousRouter",\n                {}\n            ],\n            "veh_id": "rl_3"\n        },\n        {\n            "acceleration_controller": [\n                "IDMController",\n                {\n                    "noise": 0.2\n                }\n            ],\n            "car_following_params": {\n                "controller_params": {\n                    "accel": 2.6,\n                    "carFollowModel": "IDM",\n                    "decel": 4.5,\n                    "impatience": 0.5,\n                    "maxSpeed": 30,\n                    "minGap": 0,\n                    "sigma": 0.5,\n                    "speedDev": 0.1,\n                    "speedFactor": 1.0,\n                    "tau": 1.0\n                },\n                "speed_mode": 25\n            },\n            "initial_speed": 0,\n            "lane_change_controller": [\n                "SimLaneChangeController",\n                {}\n            ],\n            "lane_change_params": {\n                "controller_params": {\n                    "laneChangeModel": "LC2013",\n                    "lcCooperative": "1.0",\n                    "lcKeepRight": "1.0",\n                    "lcSpeedGain": "1.0",\n                    "lcStrategic": "1.0"\n                },\n                "lane_change_mode": 512\n            },\n            "num_vehicles": 18,\n            "routing_controller": [\n                "ContinuousRouter",\n                {}\n            ],\n            "veh_id": "human_3"\n        }\n    ]\n}', 'run': 'PPO'}, 'evaluation_config': {}, 'evaluation_interval': None, 'evaluation_num_episodes': 10, 'gamma': 0.999, 'grad_clip': None, 'horizon': 4500, 'ignore_worker_failures': False, 'input': 'sampler', 'input_evaluation': ['is', 'wis'], 'kl_coeff': 0.2, 'kl_target': 0.02, 'lambda': 0.97, 'local_tf_session_args': {'inter_op_parallelism_threads': 8, 'intra_op_parallelism_threads': 8}, 'log_level': 'WARN', 'log_sys_usage': True, 'lr': 5e-05, 'lr_schedule': None, 'memory': 0, 'memory_per_worker': 0, 'metrics_smoothing_episodes': 100, 'min_iter_time_s': 0, 'model': {'conv_activation': 'relu', 'conv_filters': None, 'custom_action_dist': None, 'custom_model': None, 'custom_options': {}, 'custom_preprocessor': None, 'dim': 84, 'fcnet_activation': 'tanh', 'fcnet_hiddens': [32, 16], 'framestack': True, 'free_log_std': False, 'grayscale': False, 'lstm_cell_size': 256, 'lstm_use_prev_action_reward': False, 'max_seq_len': 20, 'no_final_linear': False, 'state_shape': None, 'use_lstm': False, 'vf_share_layers': True, 'zero_mean': True}, 'monitor': False, 'multiagent': {'policies': {'follower': ["<class 'ray.rllib.policy.tf_policy_template.PPOTFPolicy'>", 'Box(9,)', 'Box(1,)', {}], 'leader': [None, 'Box(9,)', 'Box(1,)', {}]}, 'policies_to_train': None, 'policy_mapping_fn': '<function policy_mapping_fn at 0x7fc13aaf1d90>'}, 'no_done_at_end': False, 'no_eager_on_workers': False, 'num_cpus_for_driver': 1, 'num_cpus_per_worker': 1, 'num_envs_per_worker': 1, 'num_gpus': 0, 'num_gpus_per_worker': 0, 'num_sgd_iter': 10, 'num_workers': 4, 'object_store_memory': 0, 'object_store_memory_per_worker': 0, 'observation_filter': 'NoFilter', 'optimizer': {}, 'output': None, 'output_compress_columns': ['obs', 'new_obs'], 'output_max_file_size': 67108864, 'postprocess_inputs': False, 'preprocessor_pref': 'deepmind', 'remote_env_batch_wait_ms': 0, 'remote_worker_envs': False, 'sample_async': False, 'sample_batch_size': 200, 'seed': None, 'sgd_minibatch_size': 128, 'shuffle_buffer_size': 0, 'shuffle_sequences': True, 'simple_optimizer': False, 'soft_horizon': False, 'synchronize_filters': True, 'tf_session_args': {'allow_soft_placement': True, 'device_count': {'CPU': 1}, 'gpu_options': {'allow_growth': True}, 'inter_op_parallelism_threads': 2, 'intra_op_parallelism_threads': 2, 'log_device_placement': False}, 'timesteps_per_iteration': 0, 'train_batch_size': 90000, 'use_gae': True, 'vf_clip_param': 10.0, 'vf_loss_coeff': 1.0, 'vf_share_layers': False}

In [ ]:
flow_params_modify = json.loads(config["env_config"]["flow_params"])

In [ ]:
flow_params_modify

In [ ]:
flow_params_modify["veh"][4]

In [ ]:
flow_params_modify["veh"][3]

In [ ]:
flow_params_modify["veh"][2]

In [ ]:
flow_params_modify["veh"][0]

In [ ]:
if config.get('multiagent', {}).get('policies', None):
    print("yes")